In [1]:
from google.colab import files
uploaded = files.upload()


Saving Raw_Tiki_reviews.csv to Raw_Tiki_reviews.csv


# Bước 1: Data Cleaning & Normalization

In [2]:
import pandas as pd
import io
import re
from google.colab import files

# Đọc file CSV với encoding utf-8-sig để xử lý cả BOM
df = pd.read_csv(io.BytesIO(uploaded['Raw_Tiki_reviews.csv']), encoding='utf-8-sig')

def standardize_date(date_str):
    """Chuẩn hóa định dạng ngày về dd/mm/yy."""
    try:
        parts = date_str.split('/')
        if len(parts) == 3:
            day, month, year = parts
            if len(year) == 4:
                year = year[2:]
            elif len(year) == 1:
                year = "0" + year
            if len(month) == 1:
                month = '0' + month
            if len(day) == 1:
                day = "0" + day
            return f"{day}/{month}/{year}"
        elif len(parts) == 2:
            day, month_year = parts
            month, year = month_year.split(' ')
            if len(year) == 4:
                year = year[2:]
            elif len(year) == 1:
                year = "0" + year
            if len(month) == 1:
                month = '0' + month
            if len(day) == 1:
                day = "0" + day
            return f"{day}/{month}/{year}"
    except (ValueError, AttributeError, IndexError):
        return None

# Áp dụng chuẩn hóa ngày
df['created_at'] = df['created_at'].apply(standardize_date)

# Điền giá trị NULL cho các ô trống
df['comment'].fillna('NULL', inplace=True)
df['images'] = df['images'].replace('[]', 'NULL')

def clean_text(text):
    """Xóa emoji và ký tự đặc biệt khỏi văn bản."""
    if pd.isna(text) or text == "NULL":
        return "NULL"
    try:
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)

        # Xóa khoảng trắng thừa
        text = ' '.join(text.split())
        return text
    except:
        return "NULL"

# Làm sạch cột comment
df['comment'] = df['comment'].apply(clean_text)

# Hiển thị kết quả
print(df)

# Xuất ra file CSV có BOM để Excel đọc đúng UTF-8
df.to_csv('tiki_reviews_cleaned.csv', index=False, encoding='utf-8-sig')


                  user_name  rating  \
0            Trần Quang Bút       5   
1             Bùi Huy Hoàng       5   
2                Hoà Nguyễn       5   
3            Ngô Thanh Tùng       5   
4        Đàm Quang Hồng Hải       5   
...                     ...     ...   
1002  Phan Quỳnh hạnh Trang       5   
1003              Chau Tran       5   
1004            Cáp Xuân Tú       5   
1005             Phuong Anh       5   
1006    Nguyễn Ngọc Bảo Hân       5   

                                                comment created_at  \
0                                                  NULL   29/04/25   
1                        Sách đẹp, giao quá nhanh 5 sao   24/04/25   
2                                                  NULL   09/05/25   
3                                                  NULL   07/05/25   
4                                                  NULL   27/04/25   
...                                                 ...        ...   
1002                                     

<ipython-input-2-7a606629f7eb>:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['comment'].fillna('NULL', inplace=True)


# Bước 2: Xử lý cột images và xuất file CSV

In [5]:
import pandas as pd
import re
from google.colab import files

# Đọc file cleaned trước đó
df = pd.read_csv('tiki_reviews_cleaned.csv', encoding='utf-8-sig')

def clean_image_field(text):
    """Loại bỏ [, ], ', và dấu phẩy thừa trong chuỗi."""
    if pd.isna(text) or text == "NULL":
        return "NULL"
    try:
        text = re.sub(r"[\[\]']", '', text)
        text = ','.join([x.strip() for x in text.split(',') if x.strip()])
        return text if text else "NULL"
    except:
        return "NULL"

# Làm sạch cột images
df['images'] = df['images'].apply(clean_image_field)

# Xuất ra 2 file: backup và bản tidy
df.to_csv('tiki_reviews_cleaned_v2.csv', index=False, encoding='utf-8-sig')
df.to_csv('tidy_reviews.csv', index=False, encoding='utf-8-sig')

# Tải file tidy về (nếu đang ở Colab)
files.download('tidy_reviews.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>